In [6]:
#設定讀取資料的年份與時段

y = 104     #110年或104年
t = '晨峰'  #晨峰或昏峰
case = '內湖轉移'  #內湖轉移或原始

In [7]:
#.xls to .csv file

import pandas as pd
import numpy as np
import os


if y == 110:
    input_file = 'TRTS4S_晨昏峰小時指派OD.xlsx'
if y == 104:
    input_file = 'TRTS4S_Y104晨昏峰小時指派OD_轉向_屏柵.xlsx'

output_file = f'{y}{t}_demand.csv'

if os.path.isfile(output_file):
    print(f"{output_file} 已存在，直接轉換為 dat 檔")
else:
    df = pd.read_excel(input_file, sheet_name=f'{y}年指派OD{t}小時')
    df.fillna(0, inplace=True)
    #save as csv
    df.to_csv(output_file, index=False, sep=',')
    print(f"Successfully saved {output_file}")

104晨峰_demand.csv 已存在，直接轉換為 dat 檔


In [ ]:
#.csv to .dat file

from tqdm import tqdm
import pandas as pd
import numpy as np

df = pd.read_csv(f'{y}{t}_demand.csv')
df.fillna(0, inplace=True)
rows = []

tamsui = [i for i in range(482,503)] #482-502

count = 0

# Modification for Beitou-Shihlin Technical Park (BSTP)
BSTP = [207,209,213]  
BSTPNeighbor = [] 
Neihu = [146,148,149,150,154,155,156,157,158,159]

# CASE: 內湖轉移
Nsum = 12350.420344126218 # Initialize Anywhere -> Neihu sum
NTransfer = 12 # Number of Neihu transfers (35,000 persons)

# CASE: 北士科就近吸引
BSTPNsum = 0 # Initialize Anywhere -> BSTP sum

#use tqdm to show the progress for 614656 rows
print("demand.dat is generating...")

for index, row in tqdm(df.iterrows(), total=len(df)):
    origin = row["I"]
    dest = row["J"]
    all = row["ALL"]
    bus = row["BUS"]
    demand = all+bus*1.8/19
    # if origin within 691 and dest within 691, then demand *= 1.5
    # if origin <691 and dest <691:
        # demand *= 1.25
    # if origin in 304-691 or dest in 304-691, then demand *= 1.5
    # if origin <= 691 and dest <= 691 and origin >= 304 and dest >= 304:
        # demand *= 1.4
    # if dest within 303, then demand *= 1.5
    if dest <= 303:
        demand *= 1
    # if origin in tamsui or dest in tamsui:
    if origin in tamsui:
        demand *= 1
    out_of_cordon=[i for i in range(692,751)]+[783,784]
    if origin not in out_of_cordon and dest not in out_of_cordon:
        rows.append({'origin':int(origin),'dest':int(dest),'demand':demand})

    if case == '內湖轉移':
        if dest in Neihu:
            Nsum += demand
            

print(f'Neihu: {Nsum}')


# print("demand.dat is generated")
# df_out = pd.DataFrame(rows)
# df_out.to_csv(f'{y}{t}_demand({case}).dat', index=False, sep='\t')
# # df_out.to_csv(f'{y}{t}_demand.dat', index=False, sep='\t')
# # df_out.to_csv(f'{y}{t}_demand.dat', index=False, sep='\t')

demand.dat is generating...


100%|██████████| 614656/614656 [00:28<00:00, 21798.12it/s]

Neihu: 12350.420344126218
